<a href="https://colab.research.google.com/github/seleznevajane/msu_ml_spring_2022/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import requests
import logging
import gzip
import json
import codecs
import re
import regex

import numpy as np
import pandas as pd

from tqdm import tqdm
from multiprocessing.dummy import Pool, Queue
from bs4 import BeautifulSoup
from time import sleep
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer 
from bs4.element import Comment

In [96]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]', 'option']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [115]:
def get_title_and_text(doc_id):
    page = open(str(doc_id) + ".dat").read()
    soup = BeautifulSoup(page, 'html.parser')

    title = soup.title.text

    txt = soup.findAll(text=True)
    visible = filter(tag_visible, txt)
    txt = u" ".join(t.strip() for t in visible)

    reg = regex.compile('[^a-zA-Zа-яА-Я ]')
    txt = reg.sub('', txt)
    if not hasattr(get_title_and_text, "stemmer"):
         get_title_and_text.stemmer = SnowballStemmer("russian")
    stop = set(stopwords.words('russian'))
    words = [stemmer.stem(word) for word in txt.split() if (not (word in stop)) and len(word) > 1]
    #words = [get_title_and_text.stemmer.stem(word) for word in txt.split()
    #         if len(word) > 3 or len(word) == 3 and word.isupper()]
    cnt = Counter(words)

    return (set(title.strip().split()), set(dict(cnt.most_common(50)).keys()))

In [ ]:
#for i in range(len(train_data)):
def process_page(pair_id):
    new_doc = train_data.iloc[pair_id]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title, text = get_title_and_text(doc_id)
    #title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, text, target))

In [ ]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, text, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist_title = []
        all_dist_text = []
        #words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, text_j, target_j = docs[j]
            #words_j = set(title_j.strip().split())
            all_dist_title.append(len(title.intersection(title_j)))
            all_dist_text.append(len(text.intersection(text_j)))
        X_train.append(sorted(all_dist_title, reverse=True)[0:15] + sorted(all_dist_text, reverse=True)[0:15])
        #X_train.append(sorted(all_dist_text, reverse=True)[0:15])
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

In [ ]:
queue = Queue()   # очередь ссылок на книги
train_data = pd.read_csv('/content/train_groups.csv')
traingroups_titledata = {}
train_data
for pair_id in len(train_data):
    queue.put(pair_id)

def process_page_wrapper(i):
    while not queue.empty():
        with gzip.open('data/part_{:05d}.jsonl.gz'.format(group_id), mode='wb') as f_json:
            f_json = codecs.getwriter('utf_16')(f_json)
            # из process_page может вылететь исключение
            page = queue.get()
            record = process_page(page)
            if record:
                record_str = json.dumps(record, indent=4, ensure_ascii=False)
                print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))
    pool.close()
    pool.join()

In [116]:
get_title_and_text(15763)

('Замена и регулировка переднего ступичного подшипника нива | Нива Ремонт',
 {'виде',
  'вынима',
  'гайк',
  'детал',
  'диск',
  'для',
  'зам',
  'замен',
  'зат',
  'ключ',
  'колес',
  'кольц',
  'комментар',
  'кулак',
  'лифт',
  'мест',
  'наружн',
  'необходим',
  'нив',
  'нижн',
  'нов',
  'образ',
  'ос',
  'отвертк',
  'откручива',
  'передн',
  'поворотн',
  'подвеск',
  'подшипник',
  'поменя',
  'послед',
  'последовательн',
  'прижимн',
  'регулировк',
  'ремонт',
  'рычаг',
  'самостоятельн',
  'сво',
  'смазк',
  'снят',
  'ступиц',
  'ступичн',
  'суппорт',
  'так',
  'тормозн',
  'узел',
  'узл',
  'чертеж',
  'чита',
  'шаров'})

In [ ]:
def group_parser(group):
    for doc_id in group:
        

In [84]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words('russian'))
print(stops)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'кто', 'теперь', 'почти', 'другой', 'тоже', 'чего', 'под', 'два', 'такой', 'по', 'про', 'разве', 'со', 'еще', 'мой', 'они', 'конечно', 'нет', 'он', 'ее', 'или', 'здесь', 'вам', 'нибудь', 'наконец', 'из', 'ему', 'раз', 'вы', 'их', 'иногда', 'более', 'до', 'тут', 'этом', 'уже', 'три', 'может', 'что', 'эти', 'потом', 'сейчас', 'чтобы', 'ней', 'во', 'перед', 'я', 'свою', 'не', 'об', 'ей', 'чуть', 'тот', 'если', 'даже', 'без', 'хоть', 'нас', 'этой', 'же', 'а', 'куда', 'после', 'тебя', 'ты', 'к', 'через', 'всегда', 'с', 'для', 'моя', 'вас', 'его', 'этого', 'всего', 'был', 'где', 'на', 'надо', 'меня', 'чтоб', 'него', 'было', 'ж', 'хорошо', 'какая', 'мы', 'этот', 'том', 'ним', 'всех', 'над', 'уж', 'можно', 'то', 'ни', 'какой', 'быть', 'но', 'тем', 'ведь', 'лучше', 'мне', 'о', 'совсем', 'них', 'только', 'была', 'чем', 'больше', 'опять', 'между', 'потому', 'сам', 'да', 'им', 'тогда', '

In [85]:
page = open("17176.dat").read().lower()# requests.get('https://gg.deals/games/')
soup = BeautifulSoup(page, 'html.parser')
page2 = open("15763.dat").read()# requests.get('https://gg.deals/games/')
soup2 = BeautifulSoup(page2, 'html.parser')

In [ ]:
soup

In [ ]:
soup2.title

<title>Замена и регулировка переднего ступичного подшипника нива | Нива Ремонт</title>

In [ ]:
txt2 = soup2.get_text(' ', strip=True)
txt2

In [97]:
txt = soup.findAll(text=True)
visible = filter(tag_visible, txt)
txt = u" ".join(t.strip() for t in visible)
txt

'          autoruk.ru                                       страница 1 из 3 замена подшипников ступицы передних колес ваз-2121  номер подшипника по каталогу 2121-3103020-10/6-2007108-а  работу проводим на смотровой канаве или подъемнике. вывешиваем переднюю часть автомобиля и снимаем колесо.       1. отсоединяем суппорт с направляющей колодок (см. разборка переднего тормозного механизма) и подвязываем их проволокой к верхнему рычагу. отворачиваем регулировочную гайку подшипников ступицы ( см. проверка и регулировка зазора в подшипниках ступицы ).      2. вынимаем конусную втулку.        3. зубилом разгибаем край стопорной пластины гайки переднего крепления рычага к поворотному кулаку. аналогично расконтриваем гайку заднего крепления.      4. накидным ключом «на 19» отворачиваем гайку заднего крепления рычага к поворотному кулаку. аналогично отворачиваем гайку переднего крепления.        5. снимаем стопорную пластину.      6. сняв рычаг с болтов, отводим его от поворотного кулака. отсое

In [86]:
txt = soup.get_text(' ', strip=True)

In [98]:
stemmer = SnowballStemmer("russian")
stop = set(stopwords.words('russian'))
words = [stemmer.stem(word) for word in txt.split() if not (word in stop)]


#for word in txt.split():
#    if len(word) > 2:
#        words.append(word)
#words = [words[i] for i in range(len(words))]


cnt = Counter(words)

In [ ]:
#stemmer = SnowballStemmer("russian")
words2 = [stemmer.stem(word) for word in txt2.split() if len(word) > 3 or len(word) == 3 and word.isupper()]
cnt2 = Counter(words2)

In [107]:
dict(cnt.most_common(50)).keys()

dict_keys(['поворотн', 'передн', 'гайк', 'рычаг', 'лад', 'подшипник', 'ступиц', 'тормозн', 'креплен', 'шаров', 'снима', 'отсоединя', 'верхн', 'отворачива', 'кулаку.', 'кулак', 'autoruk.r', '1', '3', 'колес', 'ваз-2121', '(', 'см.', ').', 'стопорн', 'аналогичн', 'задн', 'крепления.', 'кулака.', 'нижн', 'опор', 'снят', 'диском.', 'нив', 'шевролет', 'страниц', 'зам', 'номер', 'каталог', '2121-3103020-10/6-2007108-а', 'работ', 'провод', 'смотров', 'канав', 'подъемнике.', 'вывешива', 'част', 'автомобил', 'колесо.', '1.'])

In [74]:
cnt2.most_common(100)

[('подшипник', 9),
 ('нив', 8),
 ('поворотн', 8),
 ('кулак', 7),
 ('тормозн', 6),
 ('зат', 6),
 ('зам', 5),
 ('регулировк', 5),
 ('передн', 5),
 ('ступичн', 5),
 ('подвеск', 5),
 ('рычаг', 5),
 ('ступиц', 4),
 ('эт', 4),
 ('необходим', 4),
 ('самостоятельн', 4),
 ('чег', 4),
 ('лифт', 4),
 ('ключ', 4),
 ('последовательн', 4),
 ('поменя', 3),
 ('послед', 3),
 ('замен', 3),
 ('подшипник,', 3),
 ('снят', 3),
 ('суппорт', 3),
 ('нижн', 3),
 ('прижимн', 3),
 ('чита', 3),
 ('так', 3),
 ('чтоб', 3),
 ('посл', 3),
 ('откручива', 3),
 ('шаров', 3),
 ('узел', 3),
 ('наружн', 3),
 ('сво', 3),
 ('содержан', 2),
 ('звук', 2),
 ('отлич', 2),
 ('однак', 2),
 ('износ', 2),
 ('узл', 2),
 ('характерн', 2),
 ('неисправн', 2),
 ('вероятн', 2),
 ('сам', 2),
 ('настройк', 2),
 ('отсутств', 2),
 ('вывес', 2),
 ('имеет', 2),
 ('треб', 2),
 ('механизм', 2),
 ('оси.', 2),
 ('помощ', 2),
 ('аккуратн', 2),
 ('вынут', 2),
 ('полезн', 2),
 ('чертеж', 2),
 ('плоск', 2),
 ('посадочн', 2),
 ('мест', 2),
 ('образом,', 

In [ ]:
cnt.most_common(50)

[('поворотн', 8),
 ('передн', 7),
 ('.activ', 6),
 ('подшипник', 5),
 ('ступиц', 5),
 ('(adsbygoogl', 5),
 ('window.adsbygoogl', 5),
 ('гайк', 5),
 ('рычаг', 5),
 ('лад', 5),
 ('a:hover,', 4),
 ('!important;}', 4),
 ('function', 4),
 ('[]).push({});', 4),
 ('тормозн', 4),
 ('креплен', 4),
 ('шаров', 4),
 ('колес', 3),
 ('ваз-2121', 3),
 ('#0088cc;', 3),
 ('.dropdown-men', 3),
 ('a.flag', 3),
 ('снима', 3),
 ('отсоединя', 3),
 ('верхн', 3),
 ('отворачива', 3),
 ('кулаку.', 3),
 ('кулак', 3),
 ('.nav-list', 2),
 ('.nav-pills', 2),
 ('{display:non', 2),
 ('tru', 2),
 ('autoruk.r', 2),
 ('keyValu', 2),
 ('Chines', 2),
 ('w[n]', 2),
 ('стопорн', 2),
 ('аналогичн', 2),
 ('задн', 2),
 ('крепления.', 2),
 ('кулака.', 2),
 ('нижн', 2),
 ('опор', 2),
 ('снят', 2),
 ('диском.', 2),
 ('<!--', 2),
 ('//-->', 2),
 ('нив', 2),
 ('шевролет', 2),
 ('autoruk.ru/niva-vaz-2121/podveska-vaz-2121/kak-zamenit-podshipniki-stupitc-perednikh-koles-vaz-2121',
  1)]